# Step 2: Private Prediction using Syft Keras - Serving (Client)

Congratulations! After training your model with normal Keras then secured it with Syft Keras, you are ready to request some private predictions. 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## Data

Here, we just prepare our MNIST data. The pre-processing step should be the same we used during training.

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Connect to model

Before querying the model, you just have to connect to it. To do so, you can create a client. Then define the exact same three TFEWorkers (alice, bob and carol). Finally call `connect_to_model`. 

In [3]:
input_shape = (1, 28, 28, 1)
output_shape = (1, 10)

client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')

client.connect_to_model(input_shape, output_shape, alice, bob, carol)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## Query model

You are ready to get some private predictions! You just have to call `query_model`.

In [4]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [5]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1


This is great, you were able to classify correctly these three images! But what's special about these predictions is that you haven't revealed any private information to get this service. You were able to get private predictions on encrypted data with an encrypted model!